In [20]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network
from sklearn import tree
from sklearn import kernel_ridge
from sklearn import gaussian_process
from sklearn import svm

In [21]:
dfIn = pd.io.parsers.read_csv("./LibSVM_data/D-shifts_UniquePepSampled_ForPaperSCXLumos_ThermoUltimate3000_130m.standardized.csv")
dfIn.head()

,DEFeatureIdx,Peptide,Charge,SpecEValue,Channel,Quantity,Cosine,NormedApexEtDiff,Dnum,NormedRepreEt,PeptideLength,[K]Ratio,SampleName,Dnum_Standardized,NormedRepreEt_Standardized,PeptideLength_Standardized,[K]Ratio_Standardized
0,1389,EVKGDLENAFLNLVQCIQNKPLYFADR,4,2.342327e-19,1,19326970.0,0.909027,0.000095,6,1.191970,27,0.074074,DE6plex_1_1_2_NCP1_J100_20171218_2.training.tsv,-0.674119,1.886129,3.553997,0.054051
1,1389,EVKGDLENAFLNLVQCIQNKPLYFADR,4,2.342327e-19,2,17495890.0,0.908607,0.000112,12,1.191970,27,0.074074,DE6plex_1_1_2_NCP1_J100_20171218_2.training.tsv,0.088145,1.886129,3.553997,0.054051
2,1389,EVKGDLENAFLNLVQCIQNKPLYFADR,4,2.342327e-19,3,19639900.0,0.911572,0.000659,18,1.191970,27,0.074074,DE6plex_1_1_2_NCP1_J100_20171218_2.training.tsv,0.850409,1.886129,3.553997,0.054051
3,429,VPTANVSVVDLTCR,2,6.590483e-18,1,57525920.0,0.996943,0.000275,2,0.335254,14,0.000000,DE6plex_1_1_1_O_J100_NCP1_1228.training.tsv,-1.182295,-0.135586,0.003572,-1.062954
4,429,VPTANVSVVDLTCR,2,6.590483e-18,2,55259020.0,0.995364,0.000514,4,0.335254,14,0.000000,DE6plex_1_1_1_O_J100_NCP1_1228.training.tsv,-0.928207,-0.135586,0.003572,-1.062954


# Rescaling 

In [22]:
seY = dfIn['NormedApexEtDiff']
dfX = dfIn['Dnum NormedRepreEt PeptideLength [K]Ratio'.split()]

In [23]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True)
arrXScaled = min_max_scaler.fit_transform(dfX)
arrXScaled

array([[-0.83333333,  0.84539987,  0.22580645, -0.59259259],
       [-0.58333333,  0.84539987,  0.22580645, -0.59259259],
       [-0.33333333,  0.84539987,  0.22580645, -0.59259259],
       ..., 
       [-0.75      , -0.79882993, -0.87096774, -0.45      ],
       [-0.58333333, -0.79882993, -0.87096774, -0.45      ],
       [-0.25      , -0.79882993, -0.87096774, -0.45      ]])

In [24]:
print(arrXScaled.max(axis=0), arrXScaled.min(axis=0))

[ 1.  1.  1.  1.] [-1. -1. -1. -1.]


# Read Standardized

In [25]:
dfStand = dfIn['Dnum_Standardized NormedRepreEt_Standardized PeptideLength_Standardized [K]Ratio_Standardized'.split()]
arrXStand = np.array(dfStand)

In [26]:
arrXStand

array([[-0.67411876,  1.88612851,  3.55399747,  0.0540508 ],
       [ 0.08814528,  1.88612851,  3.55399747,  0.0540508 ],
       [ 0.85040932,  1.88612851,  3.55399747,  0.0540508 ],
       ..., 
       [-0.42003075, -1.39473264, -1.08886637,  0.44500249],
       [ 0.08814528, -1.39473264, -1.08886637,  0.44500249],
       [ 1.10449733, -1.39473264, -1.08886637,  0.44500249]])

# Cross Validation of Linear Model

Rescaled Data

In [27]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = linear_model.LinearRegression()
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedLMrs = np.concatenate(lPredicted)
arrTestValueLMrs = np.concatenate(lTestValue)
    
print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
np.sqrt(totalSqrError/len(seY)) * 62

Mean squared error: 0.0000032540
Root Mean squared error: 0.0018038858


0.11184092223127699

Standardized Data

In [28]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = linear_model.LinearRegression()
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedLMstd = np.concatenate(lPredicted)
arrTestValueLMstd = np.concatenate(lTestValue)
    
print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
np.sqrt(totalSqrError/len(seY)) * 62

Mean squared error: 0.0000032535
Root Mean squared error: 0.0018037567


0.11183291495068165

# Cross Validation of Polynomial Linear Model

Rescaled Data

In [29]:
poly = preprocessing.PolynomialFeatures(2)
arrXPoly = poly.fit_transform(arrXScaled)
arrXPoly

array([[ 1.        , -0.83333333,  0.84539987, ...,  0.05098855,
        -0.13381123,  0.35116598],
       [ 1.        , -0.58333333,  0.84539987, ...,  0.05098855,
        -0.13381123,  0.35116598],
       [ 1.        , -0.33333333,  0.84539987, ...,  0.05098855,
        -0.13381123,  0.35116598],
       ..., 
       [ 1.        , -0.75      , -0.79882993, ...,  0.75858481,
         0.39193548,  0.2025    ],
       [ 1.        , -0.58333333, -0.79882993, ...,  0.75858481,
         0.39193548,  0.2025    ],
       [ 1.        , -0.25      , -0.79882993, ...,  0.75858481,
         0.39193548,  0.2025    ]])

In [30]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXPoly):
    reg = linear_model.LinearRegression()
    linearModel = reg.fit(arrXPoly[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXPoly[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedPLMrs = np.concatenate(lPredicted)
arrTestValuePLMrs = np.concatenate(lTestValue)
print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
np.sqrt(totalSqrError/len(seY)) * 62

Mean squared error: 0.0000030030
Root Mean squared error: 0.0017329048


0.10744009677796136

Standardized Data

In [31]:
poly = preprocessing.PolynomialFeatures(2)
arrXPoly = poly.fit_transform(dfStand)
arrXPoly

array([[  1.00000000e+00,  -6.74118763e-01,   1.88612851e+00, ...,
          1.26308980e+01,   1.92096401e-01,   2.92148881e-03],
       [  1.00000000e+00,   8.81452769e-02,   1.88612851e+00, ...,
          1.26308980e+01,   1.92096401e-01,   2.92148881e-03],
       [  1.00000000e+00,   8.50409317e-01,   1.88612851e+00, ...,
          1.26308980e+01,   1.92096401e-01,   2.92148881e-03],
       ..., 
       [  1.00000000e+00,  -4.20030750e-01,  -1.39473264e+00, ...,
          1.18562997e+00,  -4.84548247e-01,   1.98027218e-01],
       [  1.00000000e+00,   8.81452769e-02,  -1.39473264e+00, ...,
          1.18562997e+00,  -4.84548247e-01,   1.98027218e-01],
       [  1.00000000e+00,   1.10449733e+00,  -1.39473264e+00, ...,
          1.18562997e+00,  -4.84548247e-01,   1.98027218e-01]])

In [32]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXPoly):
    reg = linear_model.LinearRegression()
    linearModel = reg.fit(arrXPoly[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXPoly[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedPLMstd = np.concatenate(lPredicted)
arrTestValuePLMstd = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000030027
Root Mean squared error: 0.0017328164
0.107434619756


# Cross Validation of KNN

Rescaled Data

In [33]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = neighbors.KNeighborsRegressor(n_neighbors=30)
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKNNrs = np.concatenate(lPredicted)
arrTestValueKNNrs = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000027854
Root Mean squared error: 0.0016689379
0.103474149613


Standardized Data

In [34]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = neighbors.KNeighborsRegressor(n_neighbors=30)
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKNNstd = np.concatenate(lPredicted)
arrTestValueKNNstd = np.concatenate(lTestValue)


print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000028241
Root Mean squared error: 0.0016804913
0.104190458853


Rescaled, Polynomial Data

In [35]:
poly = preprocessing.PolynomialFeatures(2)
arrXPoly = poly.fit_transform(arrXScaled)

lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXPoly):
    reg = neighbors.KNeighborsRegressor(n_neighbors=30)
    linearModel = reg.fit(arrXPoly[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXPoly[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKNNrsP = np.concatenate(lPredicted)
arrTestValueKNNrsP = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000027888
Root Mean squared error: 0.0016699777
0.10353861967


Standardized, Polynomial Data

In [36]:
poly = preprocessing.PolynomialFeatures(2)
arrXPoly = poly.fit_transform(dfStand)

lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXPoly):
    reg = neighbors.KNeighborsRegressor(n_neighbors=30)
    linearModel = reg.fit(arrXPoly[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXPoly[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKNNstdP = np.concatenate(lPredicted)
arrTestValueKNNstdP = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000028514
Root Mean squared error: 0.0016886215
0.104694534165


# Cross Validation of RadiusNeighborsRegressor

Rescaled Data

In [37]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = neighbors.RadiusNeighborsRegressor()
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedRNRrs = np.concatenate(lPredicted)
arrTestValueRNRrs = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000038895
Root Mean squared error: 0.0019721919
0.122275896585


Standardized Data

In [38]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = neighbors.RadiusNeighborsRegressor(radius=40)
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedRNRstd = np.concatenate(lPredicted)
arrTestValueRNRstd = np.concatenate(lTestValue)


print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000046668
Root Mean squared error: 0.0021602678
0.133936604344


# Cross Validation of Multi-Layer Perceptron

Rescaled Data

In [39]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = neural_network.MLPRegressor(hidden_layer_sizes=(400, 400, 400),)
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedMLPrs = np.concatenate(lPredicted)
arrTestValueMLPrs = np.concatenate(lTestValue)


print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000035179
Root Mean squared error: 0.0018755969
0.116287007385


Standardized Data

In [40]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = neural_network.MLPRegressor(hidden_layer_sizes=(400, 400, 400),)
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedMLPstd = np.concatenate(lPredicted)
arrTestValueMLPstd = np.concatenate(lTestValue)


print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000059064
Root Mean squared error: 0.0024303055
0.150678939276


# Cross Validation of Decision Tree

Rescaled Data

In [41]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = tree.DecisionTreeRegressor()
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedDTrs = np.concatenate(lPredicted)
arrTestValueDTrs = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000048596
Root Mean squared error: 0.0022044592
0.136676469855


Standardized Data

In [42]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = tree.DecisionTreeRegressor()
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedDTstd = np.concatenate(lPredicted)
arrTestValueDTstd = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000050021
Root Mean squared error: 0.0022365272
0.138664684443


# Cross Validation of Gaussian Process

Rescaled Data

In [43]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = gaussian_process.GaussianProcessRegressor()
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedGPrs = np.concatenate(lPredicted)
arrTestValueGPrs = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.0000472519
Root Mean squared error: 0.0068740045
0.42618828147


Standardized Data

In [44]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = gaussian_process.GaussianProcessRegressor()
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedGPstd = np.concatenate(lPredicted)
arrTestValueGPstd = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Mean squared error: 0.1657577260
Root Mean squared error: 0.4071335482
25.242279986


# Cross Validation of KernelRidgeRegression

Rescaled Data

In [ ]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = kernel_ridge.KernelRidge(kernel='rbf', gamma='auto')
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKRRrs = np.concatenate(lPredicted)
arrTestValueKRRrs = np.concatenate(lTestValue)


print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Standardized Data

In [ ]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    reg = kernel_ridge.KernelRidge(kernel='rbf', gamma='auto')
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedKRRstd = np.concatenate(lPredicted)
arrTestValueKRRstd = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

# Cross Validation of Nu-SVR

Rescaled Data

In [ ]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXScaled):
    reg = svm.NuSVR(kernel='rbf', tol=0.00005, nu=0.5, C=1.0, gamma='auto', cache_size=20000)
    linearModel = reg.fit(arrXScaled[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXScaled[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedNuSVRrs = np.concatenate(lPredicted)
arrTestValueNuSVRrs = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)

Standardized Data

In [ ]:
lPredicted = []
lTestValue = []

kf = KFold(n_splits=10, shuffle=True)
totalSqrError = 0
for train_idx, test_idx in kf.split(arrXStand):
    
    reg = svm.NuSVR(kernel='rbf', tol=0.00005, nu=0.5, C=0.1, gamma='auto', cache_size=20000)
    linearModel = reg.fit(arrXStand[train_idx], seY[train_idx])
    arrTestPredicted = reg.predict(arrXStand[test_idx])
    totalSqrError += np.sum((arrTestPredicted - seY[test_idx]) ** 2)
    
    lPredicted.append(arrTestPredicted)
    lTestValue.append(seY[test_idx])
    
arrPredictedNuSVRstdNu = np.concatenate(lPredicted)
arrTestValueNuSVRstdNu = np.concatenate(lTestValue)

print("Mean squared error: %.10f"%(totalSqrError/len(seY)))
print("Root Mean squared error: %.10f"% np.sqrt(totalSqrError/len(seY)))
print(np.sqrt(totalSqrError/len(seY)) * 62)


# Dump to pickle

In [ ]:
with open('CV_results_Regression_models.pk', 'wb') as pkfile:
    pickle.dump({'arrPredictedLMrs': arrPredictedLMrs,
                 'arrTestValueLMrs': arrTestValueLMrs,
                 'arrPredictedLMstd': arrPredictedLMstd,
                 'arrTestValueLMstd': arrTestValueLMstd,
                 
                 'arrPredictedPLMrs': arrPredictedPLMrs,
                 'arrTestValuePLMrs': arrTestValuePLMrs,
                 'arrPredictedPLMstd': arrPredictedPLMstd,
                 'arrTestValuePLMstd': arrTestValuePLMstd,
                 
                 'arrPredictedKNNrs': arrPredictedKNNrs,
                 'arrTestValueKNNrs': arrTestValueKNNrs,
                 'arrPredictedKNNstd': arrPredictedKNNstd,
                 'arrTestValueKNNstd': arrTestValueKNNstd,
                 
                 'arrPredictedKNNrsP': arrPredictedKNNrsP,
                 'arrTestValueKNNrsP': arrTestValueKNNrsP,
                 'arrPredictedKNNstdP': arrPredictedKNNstdP,
                 'arrTestValueKNNstdP': arrTestValueKNNstdP,
                 
                 'arrPredictedRNRrs': arrPredictedRNRrs,
                 'arrTestValueRNRrs': arrTestValueRNRrs,
                 'arrPredictedRNRstd': arrPredictedRNRstd,
                 'arrTestValueRNRstd': arrTestValueRNRstd,
                 
                 'arrPredictedMLPrs': arrPredictedMLPrs,
                 'arrTestValueMLPrs': arrTestValueMLPrs,
                 'arrPredictedMLPstd': arrPredictedMLPstd,
                 'arrTestValueMLPstd': arrTestValueMLPstd,
                 
                 'arrPredictedDTrs': arrPredictedDTrs,
                 'arrTestValueDTrs': arrTestValueDTrs,
                 'arrPredictedDTstd': arrPredictedDTstd,
                 'arrTestValueDTstd': arrTestValueDTstd,
                 
                 'arrPredictedGPrs': arrPredictedGPrs,
                 'arrTestValueGPrs': arrTestValueGPrs,
                 'arrPredictedGPstd': arrPredictedGPstd,
                 'arrTestValueGPstd': arrTestValueGPstd,
                 
                 'arrPredictedKRRrs': arrPredictedKRRrs,
                 'arrTestValueKRRrs': arrTestValueKRRrs,
                 'arrPredictedKRRstd': arrPredictedKRRstd,
                 'arrTestValueKRRstd': arrTestValueKRRstd,
                 
                 'arrPredictedNuSVRrs': arrPredictedNuSVRrs,
                 'arrTestValueNuSVRrs': arrTestValueNuSVRrs,
                 'arrPredictedNuSVRstd': arrPredictedNuSVRstd,
                 'arrTestValueNuSVRstd': arrTestValueNuSVRstd,
                 
                },
                pkfile)
    


# Load From Pickle

In [ ]:
with open('CV_results_Regression_models.pk', 'rb') as pkfile:
    dArr = pickle.load(pkfile)

# Benchmark of Each Model

In [ ]:
def draw_benchmark(model):
    arrPredicted = dArr['arrPredicted' + model]
    arrTestValue = dArr['arrTestValue' + model]
    draw_benchmark_fromArr(arrPredicted, arrTestValue, model)
    
def draw_benchmark_fromArr(arrPredicted, arrTestValue, model=None):
    rmse = np.sqrt(np.mean(((arrPredicted - arrTestValue) ** 2)))
    corr = stats.pearsonr(arrPredicted, arrTestValue)[0]

    print('RMSE: %.5e'%rmse)
    print('Corr: %.5f'%corr)

    left = 0.2
    bottom = 0.2
    width = 0.7
    height = 0.7
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_axes([left, bottom, width, height]) 

    
    ax.set_xlim((-0.003, 0.008))
    ax.set_ylim((-0.003, 0.008))
    ax.set_xlabel('Predicted RT shift')
    ax.set_ylabel('Real RT shift')
    ax.grid()

    ax.scatter(arrPredicted, arrTestValue,
               alpha=0.02, color='k', marker='o', rasterized=True)

    if model is not None:
        plt.savefig('Predicted_RT_shift_vs_Real_RT_shift_{}.pdf'.format(model), format='pdf', dpi=500)

In [ ]:
#Linear Model, Rescaled
draw_benchmark('LMrs')

In [ ]:
#Linear Model, Standardized
draw_benchmark('LMstd')

In [ ]:
#Polynomial Linear Model, Rescaled
draw_benchmark('PLMrs')

In [ ]:
#Polynomial Linear Model, standardized
draw_benchmark('PLMstd')

In [ ]:
#KNN, Rescaled
draw_benchmark('KNNrs')

In [ ]:
#KNN, Standardized
draw_benchmark('KNNstd')

In [ ]:
#KNN, Rescaled, Polynomial Data
draw_benchmark('KNNrsP')

In [ ]:
#KNN, Standardized, Polynomial Data
draw_benchmark('KNNstdP')

In [ ]:
#Radius Neighbor Regressor, rescaled
draw_benchmark('RNRrs')

In [ ]:
#Radius Neighbor Regressor, standardized
draw_benchmark('RNRstd')

In [ ]:
#Multi Layer Perceptron, rescaled
draw_benchmark('MLPrs')

In [ ]:
#Multi Layer Perceptron, std
draw_benchmark('MLPstd')

In [ ]:
#Decision Tree, Rescaled
draw_benchmark('DTrs')

In [ ]:
#Decision Tree, Standardized
draw_benchmark('DTstd')

In [ ]:
#Gaussian Process, Rescaled
draw_benchmark('GPrs')

In [ ]:
#Gaussian Process, Standardized
draw_benchmark('GPstd')

In [ ]:
#KernalRidgeRegression, Rescaled
draw_benchmark('KRRrs')

In [ ]:
#KernalRidgeRegression, Rescaled
draw_benchmark('KRRstd')

In [ ]:
#NuSVR, Rescaled
draw_benchmark('NuSVRrs')

In [ ]:
#NuSVR, Standardized
draw_benchmark('NuSVRstd')

In [55]:
#NuSVR, Rescaled, gamma=1
#draw_benchmark('NuSVRrsg1')

In [56]:
#NuSVR, Standardized, gamma=1
#draw_benchmark('NuSVRstdg1')

In [11]:
svr = svm.NuSVR(kernel='rbf', cache_size=20000)
params = {"tol": [0.0005, 0.0001, 0.00005],
          "nu":[0.7, 0.5, 0.3], 
          "C":[0.01, 0.1, 1, 10],
          "gamma": [0.001, 0.01, 0.1, 0.2, 1][::-1]}

grid_search = GridSearchCV(svr, params, n_jobs=32, scoring='r2', verbose=2, cv=5)
grid_search.fit(arrXStand, seY)

print("Best parameters set found on development set:")
print()
print(grid_search.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in grid_search.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=1 ..............................
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=1 .............................
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=1 ..............................
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=1 ..............................
[CV] nu=0.7, t

[CV] nu=0.5, tol=5e-05, C=0.01, gamma=0.2 ............................
[CV] ............... nu=0.3, tol=5e-05, C=0.01, gamma=1, total=  12.0s
[CV] nu=0.5, tol=5e-05, C=0.01, gamma=0.2 ............................
[CV] ............... nu=0.3, tol=5e-05, C=0.01, gamma=1, total=  12.0s
[CV] nu=0.3, tol=0.0005, C=0.01, gamma=0.2 ...........................
[CV] ............... nu=0.3, tol=5e-05, C=0.01, gamma=1, total=  12.7s
[CV] nu=0.3, tol=0.0005, C=0.01, gamma=0.2 ...........................
[CV] ............ nu=0.7, tol=0.0005, C=0.01, gamma=0.2, total=  16.7s
[CV] nu=0.3, tol=0.0005, C=0.01, gamma=0.2 ...........................
[CV] ............ nu=0.7, tol=0.0005, C=0.01, gamma=0.2, total=  16.2s
[CV] nu=0.3, tol=0.0005, C=0.01, gamma=0.2 ...........................
[CV] ............ nu=0.7, tol=0.0005, C=0.01, gamma=0.2, total=  16.6s
[CV] nu=0.3, tol=0.0005, C=0.01, gamma=0.2 ...........................
[CV] ............ nu=0.7, tol=0.0005, C=0.01, gamma=0.2, total=  17.0s
[CV] .

[Parallel(n_jobs=32)]: Done  98 tasks      | elapsed:  1.3min


[CV] ............ nu=0.5, tol=0.0001, C=0.01, gamma=0.1, total=  12.9s
[CV] nu=0.3, tol=5e-05, C=0.01, gamma=0.1 ............................
[CV] ............ nu=0.5, tol=0.0001, C=0.01, gamma=0.1, total=  13.2s
[CV] nu=0.3, tol=5e-05, C=0.01, gamma=0.1 ............................
[CV] ............ nu=0.3, tol=0.0005, C=0.01, gamma=0.1, total=   8.9s
[CV] nu=0.3, tol=5e-05, C=0.01, gamma=0.1 ............................
[CV] ............ nu=0.3, tol=0.0005, C=0.01, gamma=0.1, total=   9.3s
[CV] nu=0.3, tol=5e-05, C=0.01, gamma=0.1 ............................
[CV] ............ nu=0.7, tol=0.0005, C=0.01, gamma=0.1, total=  15.8s
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=0.01 ..........................
[CV] ............. nu=0.7, tol=5e-05, C=0.01, gamma=0.1, total=  16.3s
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=0.01 ..........................
[CV] ............ nu=0.5, tol=0.0001, C=0.01, gamma=0.1, total=  14.3s
[CV] nu=0.7, tol=0.0005, C=0.01, gamma=0.01 ..........................
[CV] .

[CV] ........... nu=0.5, tol=0.0001, C=0.01, gamma=0.01, total=  11.8s
[CV] nu=0.7, tol=0.0001, C=0.01, gamma=0.001 .........................
[CV] ........... nu=0.5, tol=0.0001, C=0.01, gamma=0.01, total=  11.1s
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=0.001 ..........................
[CV] ............ nu=0.5, tol=5e-05, C=0.01, gamma=0.01, total=  10.3s
[CV] ........... nu=0.5, tol=0.0001, C=0.01, gamma=0.01, total=  11.4s
[CV] ............ nu=0.7, tol=5e-05, C=0.01, gamma=0.01, total=  14.3s
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=0.001 ..........................
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=0.001 ..........................
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=0.001 ..........................
[CV] ........... nu=0.5, tol=0.0001, C=0.01, gamma=0.01, total=  10.9s
[CV] nu=0.7, tol=5e-05, C=0.01, gamma=0.001 ..........................
[CV] ............ nu=0.5, tol=5e-05, C=0.01, gamma=0.01, total=  11.2s
[CV] nu=0.5, tol=0.0005, C=0.01, gamma=0.001 .........................
[CV] .

...

[CV] ............. nu=0.5, tol=0.0005, C=10, gamma=0.01, total= 3.1min
[CV] nu=0.3, tol=0.0001, C=10, gamma=0.01 ............................
[CV] ............... nu=0.5, tol=5e-05, C=10, gamma=0.1, total=20.7min
[CV] nu=0.3, tol=0.0001, C=10, gamma=0.01 ............................
[CV] ............... nu=0.5, tol=5e-05, C=10, gamma=0.1, total=20.4min
[CV] nu=0.3, tol=5e-05, C=10, gamma=0.01 .............................
[CV] .............. nu=0.7, tol=5e-05, C=10, gamma=0.01, total= 4.9min
[CV] nu=0.3, tol=5e-05, C=10, gamma=0.01 .............................
[CV] .............. nu=0.7, tol=5e-05, C=10, gamma=0.01, total= 5.0min
[CV] nu=0.3, tol=5e-05, C=10, gamma=0.01 .............................
[CV] ............... nu=0.5, tol=5e-05, C=10, gamma=0.1, total=20.4min
[CV] nu=0.3, tol=5e-05, C=10, gamma=0.01 .............................
[CV] .............. nu=0.7, tol=5e-05, C=10, gamma=0.01, total= 4.9min
[CV] nu=0.3, tol=5e-05, C=10, gamma=0.01 .............................
[CV] .

[CV] .............. nu=0.5, tol=5e-05, C=10, gamma=0.01, total= 3.7min
[CV] ............ nu=0.5, tol=0.0001, C=10, gamma=0.001, total=  40.7s
[CV] ............ nu=0.3, tol=0.0005, C=10, gamma=0.001, total=  24.4s
[CV] ............ nu=0.3, tol=0.0001, C=10, gamma=0.001, total=  24.7s
[CV] .............. nu=0.3, tol=5e-05, C=10, gamma=0.01, total= 2.4min
[CV] ............ nu=0.3, tol=0.0001, C=10, gamma=0.001, total=  25.7s
[CV] ............ nu=0.5, tol=0.0001, C=10, gamma=0.001, total=  39.5s
[CV] ............ nu=0.3, tol=0.0001, C=10, gamma=0.001, total=  23.7s
[CV] ............. nu=0.5, tol=5e-05, C=10, gamma=0.001, total=  36.7s
[CV] ............ nu=0.3, tol=0.0001, C=10, gamma=0.001, total=  23.7s
[CV] ............. nu=0.5, tol=5e-05, C=10, gamma=0.001, total=  39.2s
[CV] ............. nu=0.3, tol=5e-05, C=10, gamma=0.001, total=  23.3s
[CV] ............. nu=0.5, tol=5e-05, C=10, gamma=0.001, total=  38.1s
[CV] ............ nu=0.3, tol=0.0001, C=10, gamma=0.001, total=  24.3s
[CV] .

[Parallel(n_jobs=32)]: Done 900 out of 900 | elapsed: 196.0min finished


Best parameters set found on development set:

{'tol': 5e-05, 'nu': 0.5, 'gamma': 1, 'C': 0.01}

Grid scores on development set:

0.377 (+/-0.073) for {'tol': 0.0005, 'nu': 0.7, 'gamma': 1, 'C': 0.01}
0.378 (+/-0.071) for {'tol': 0.0001, 'nu': 0.7, 'gamma': 1, 'C': 0.01}
0.378 (+/-0.071) for {'tol': 5e-05, 'nu': 0.7, 'gamma': 1, 'C': 0.01}
0.378 (+/-0.072) for {'tol': 0.0005, 'nu': 0.5, 'gamma': 1, 'C': 0.01}
0.380 (+/-0.072) for {'tol': 0.0001, 'nu': 0.5, 'gamma': 1, 'C': 0.01}
0.380 (+/-0.072) for {'tol': 5e-05, 'nu': 0.5, 'gamma': 1, 'C': 0.01}
0.378 (+/-0.064) for {'tol': 0.0005, 'nu': 0.3, 'gamma': 1, 'C': 0.01}
0.376 (+/-0.066) for {'tol': 0.0001, 'nu': 0.3, 'gamma': 1, 'C': 0.01}
0.376 (+/-0.065) for {'tol': 5e-05, 'nu': 0.3, 'gamma': 1, 'C': 0.01}
0.376 (+/-0.061) for {'tol': 0.0005, 'nu': 0.7, 'gamma': 0.2, 'C': 0.01}
0.379 (+/-0.062) for {'tol': 0.0001, 'nu': 0.7, 'gamma': 0.2, 'C': 0.01}
0.379 (+/-0.062) for {'tol': 5e-05, 'nu': 0.7, 'gamma': 0.2, 'C': 0.01}
0.378 (+/-0.061)

/casa/yeon/.local/lib/python3.5/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [14]:
grid_search.best_params_

{'C': 0.01, 'gamma': 1, 'nu': 0.5, 'tol': 5e-05}

In [17]:
(grid_search.best_score_)**0.5

0.61669706572788863